## Лабораторная работа 7.
--------

Выполнил: Маковеев Никита Владимирович.

--------

# Рекомендательные системы

Идея рекомендательных систем на основе ближайшего соседа – использовать предпочтения некоторой группы людей, похожих на данного пользователя, чтобы рекомендовать ему что-либо.

У такой модели много применений:
    
   * рекомендация товаров интернет-магазинов
   * рекомендация музыки или фильмов
   * рекомендация людей/контента в социальных сетях

<a href="https://github.com/gustavohpo2016/notebook/blob/master/Rec_Engine_Image_Amazon.png?raw=true" target="_blank"><img src="https://github.com/gustavohpo2016/notebook/blob/master/Rec_Engine_Image_Amazon.png?raw=true"
alt="IMAGE ALT TEXT HERE" width="700" border="0" /></a>



**Пример рекомендательной системы Amazon.com**

<a href="https://habrastorage.org/r/w1560/files/196/e81/3fe/196e813fe26045efba660e09de211446.png" target="_blank"><img src="https://habrastorage.org/r/w1560/files/196/e81/3fe/196e813fe26045efba660e09de211446.png"
  alt="IMAGE ALT TEXT HERE" width="700" border="0" /></a>


Наиболее популярными решениями для построения рекомендальных систем являются: user-based (оценивается схожесть между пользователями) и item-based (оценивается схожесть между объектами).

<a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" target="_blank"><img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png"
alt="IMAGE ALT TEXT HERE" width="700" border="0" /></a>



**Рекомендация фильмов от IMDb**

Естественный способ получить рекомендацию о чем-либо - спросить мнение об этом у друзей или любых людей, которым нравится то же, что и вам. Эту идею можно использовать и для машины: для каждого человека алгоритм просматривает большую группу людей и ищет в ней подгруппу с похожим на данного человека вкусом. Далее создается список того, что еще нравится этим людям, а затем человеку рекомендуются предложения из этого списка. Такой алгоритм называется алгоритмом **коллаборативной фильтрации**.

**Отключаем предупреждения**

Библиотека **warnings** отвечает за то, какие предупреждения (warnings) о работе будут выводиться пользователю.
FutureWarning - предупреждения о том, как изменится работа библиотек в будущих версиях. Такие предупреждения мы будем игнорировать.
Чтобы включить режим игнорирования, мы отбираем все предупреждения из категории FutureWarning и выбираем для них действия 'ignore'.
Это делается вызовом функции simplefilter c задание двух атрибутов: действия action и категории предупреждений category.

**Code 0:**

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Для красивого вывода на экран сложных объектов будем использовать **pprint**

In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

### Подготовка данных

Чтобы хранить сразу много предпочтений для каждого человека, удобнее всего воспользоваться **вложенным словарем**.

Самый простой способ создать словарь в python - использовать фигурные скобки {}. Данные в словаре хранятся в формате ключ – значение, разделенные двоеточием:

```python
dict = {'key': 'value'}
```

Мы будем работать с *вложенным словарем* кинокритиков и выставленных ими оценок для небольшого набора данных о фильмах. Построим его следующим образом:

1) для каждого кинокритика создаем словарь оценок фильмов в формате
   ```python
    scores_dict = {
        'film_1': 'score_1',
        'film_2': 'score_2',
        ...
    }
   ```
   
2) создаем словарь кинокритиков, где в качестве значений будет соответствующий ему словарь оценок
   ```python
     critics = {
         'name_1': 'scores_dict_1',
         'name_2': 'scores_dict_2',
         ...
     }
   ```
   
**Code 1:**

In [3]:
# Словарь кинокритиков и выставленных ими оценок
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 3.5},
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0,
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0},
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [4]:
pp.pprint(critics)

{   'Claudia Puig': {   'Just My Luck': 3.0,
                        'Snakes on a Plane': 3.5,
                        'Superman Returns': 4.0,
                        'The Night Listener': 4.5,
                        'You, Me and Dupree': 2.5},
    'Gene Seymour': {   'Just My Luck': 1.5,
                        'Lady in the Water': 3.0,
                        'Snakes on a Plane': 3.5,
                        'Superman Returns': 5.0,
                        'The Night Listener': 3.0,
                        'You, Me and Dupree': 3.5},
    'Jack Matthews': {   'Lady in the Water': 3.0,
                         'Snakes on a Plane': 4.0,
                         'Superman Returns': 5.0,
                         'The Night Listener': 3.0,
                         'You, Me and Dupree': 3.5},
    'Lisa Rose': {   'Just My Luck': 3.0,
                     'Lady in the Water': 2.5,
                     'Snakes on a Plane': 3.5,
                     'Superman Returns': 3.5,
                 

**Code 2:**  
**Упражнение.** Выведите оценки, которые выставил Toby

In [5]:
# Самопроверка 2. Впишите код ниже и получите ответ аналогичный представленному выше

# У вас должно получиться:
answer = {'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}
answer_toby = critics['Toby']
pp.pprint(answer_toby)

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}


**Code 3:**  
**Упражнение.** Выведите оценку критика Lisa Rose для фильма Lady in the Water

In [6]:
# Самопроверка 3. Впишите код ниже и получите ответ аналогичный представленному выше

answer_lisa = critics['Lisa Rose']
LW_rating = answer_lisa['Lady in the Water']
LW_rating


2.5

### Критерий похожести

Как мы уже говорили, чтобы выделить подгруппу людей с похожим вкусом, необходимо как-то определить, насколько люди похожи. В нашем случае мы будем сравнивать оценки людей у одинаковых фильмов. Мы рассмотрим два способа сравнения. С помощью
- расстояния Евклида
- корреляции Пирсона

**Оценка по евклидову расстоянию**

Евклидово расстояние для точек $x = (x_1, x_2, ... , x_n)$ и $y = (y_1, y_2, ... , y_n)$ определяется следующим образом:

\begin{equation}
d(x, y) = \sqrt{\sum_{i = 1}^n (x_i - y_i)^2}
\end{equation}

Для вычисления квадратного корня можно воспользоваться готовой функцией **sqrt()** из библиотеки для работы с числами **math**:

**Code 4:**

In [7]:
from math import sqrt

Для понимания расстояния Евклида рассмотрим случай, когда *n = 2*. То есть каждый объект выборки (точка) описывается двумя параметрами (координатами). $A = (x_1, y_1)$ и $B = (x_2, y_2)$. Тогда Евклидово расстояние равно расстоянию между точками на плоскости.

\begin{equation}
d(A,B) = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
\end{equation}

Или, если представить задачу геометрически:

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://zaochnik.com/uploads/2018/01/08/image030.png"
alt="IMAGE ALT TEXT HERE" width="300" border="0" /></a>


**Упражнение**  
Реализуйте функцию для вычисления расстояния Евклида

In [8]:
def compute_euclid(x, y):
    return (sqrt((x[0]-y[0])**2 + (x[1]-y[1])**2))


Для примера вычислим расстояния Евклида для близких точек с координатам (1, 1) и (2, 3):

**Code 5:**

In [9]:
# 4. TODO
compute_euclid([1, 1], [2, 3])

# должно получиться:  2.23606797749979

2.23606797749979

Теперь вычислим расстояния Евклида для точек, которые находятся далеко (1, 1) и (8, 10):

**Code 6:**

In [10]:
# 5. TODO
compute_euclid([1, 1], [8, 10])

# должно получиться: 11.40175425099138

11.40175425099138

Расстояние, вычисленное по этой формуле, будет тем меньше, чем больше сходство людей (чем ближе точки). Нам же нужна функция, значение которой будет, наоборот, большое, если люди сильно похожи друг на друга. То есть схожесть больше (от 0 до 1), если точки ближе друг другу


Для этого будем использовать функцию "похожести" в таком виде:

\begin{equation}
\text{similarity }(x, y) = \dfrac{1}{1 + d(x, y)},
\end{equation}

где $d(x, y)$ - расстояние Евклида.

Пример вычисления сходства по новой функции для точек с координатами (1, 1) и (2, 3):

**Code 7:**

In [11]:
# 6. TODO
# Должно получиться 0.309016

1 / (1 + compute_euclid([1, 1], [2, 3]))


0.3090169943749474

Пример вычисления сходства по новой функции для точек с координатами (1, 1) и (8, 10):

**Code 8:**

In [12]:
# 7. TODO
# По аналогии с предыдущей формулой вычислите сходство по новой функции для точек с координатами (1, 1) и (8, 10)
answer = 1 / (1 + compute_euclid([1, 1], [8, 10]))
answer


# Должно получиться: 0.08063375388365411

0.08063375388365411

Далее реализуем несколько простых функций для коллаборативной фильтрации на основе ближайшего соседа. Чтобы воспользоваться ими, необходимо их импортировать из модуля **recommendations**:

**Code 9:**

In [13]:
%%capture
!wget https://www.dropbox.com/s/udwga4a1tohd99b/recommendations.py

In [14]:
from recommendations import *

Функцию для подсчета сходства двух критиков на основе расстояния Евклида реализована в модуле **recommendations** и называется **sim_distance()**. Функция принимает на вход три аргумента:
* словарь критиков
* имя первого критика для сравнения
* имя второго критика для сравнения

Посмотрим, как работает эта функция для двух критиков Lisa Rose и Toby:

**Code 10:**

In [15]:
sim_distance(critics, "Lisa Rose", "Toby")

0.2222222222222222

**Упражнение**  
Реализуйте функцию для вычисления расстояния sim_distance для двух критиков: "Mick LaSalle" и "Gene Seymour"

In [16]:
# 9. TODO
sim_distance(critics, "Mick LaSalle", "Gene Seymour")

# Должно получиться 0.12903

0.12903225806451613

Эта функция работает также как:

\begin{equation}
\text{similarity }(x, y) = \dfrac{1}{1 + d(x, y)},
\end{equation}

где $d(x, y)$ - расстояние Евклида.

Однако, по осям будет фильмы и оценки от критиков. Если у критиков нет одинаковых фильмов, то мы этих критиков не рассматриваем

**Коэффициент корреляции Пирсона**

Другой способ определять степень схожести - использовать коэффициент корреляции Пирсона. Этот коэффициент показывает, насколько два набора данных хорошо ложатся на прямую.

<a href="https://drive.google.com/uc?id=1qXZxG2sE3Y0oAcgayaEdN6nCYVunj_83" target="_blank"><img src="https://drive.google.com/uc?id=1qXZxG2sE3Y0oAcgayaEdN6nCYVunj_83"
alt="IMAGE ALT TEXT HERE" width="800" border="0" /></a>

Два кинокритика со средним значением корреляции 0.41 (слева) и два кинокритика с высоким значением корреляции 0.75 (справа).  

Корреляция Пирсона считается по формуле:
    
\begin{equation}
    \text{corr}(x, y) = \dfrac{\sum_{i = 1}^n (x_i - \frac{\sum_{j = 1}^n x_j}{n}) (y_i - \frac{\sum_{j = 1}^n y_j}{n})}{\sqrt{\sum_{i = 1}^n (x_i - \frac{\sum_{j = 1}^n x_j}{n})^2 \sum_{i = 1}^n (y_i - \frac{\sum_{j = 1}^n y_j}{n})^2}}
\end{equation}

По аналогии с функцией **sim_distance()** в модуле **recommendations** реализована функция для подсчета сходства двух критиков на основе корреляции Пирсона. Она называется **sim_pearson()**. Функция принимает на вход те же самые аргументы, что и **sim_distance()**:
* словарь критиков
* имя первого критика для сравнения
* имя второго критика для сравнения

Посмотрим, как работает эта функция для двух критиков Lisa Rose и Gene Seymour:

**Code 11:**  
**Задание.** Допишите функцию для вычисления корреляции Пирсона

In [17]:
def pearson_corr(x, y):
    res = 0
    n = len(x)
    mean_x=sum(x)/n
    mean_y = sum(y)/n
    s, s_x2, s_y2 = 0, 0, 0
    for i in range(n):
        s += (x[i]-mean_x)*(y[i]-mean_y)
        s_x2 += (x[i]-mean_x)**2
        s_y2 += (y[i]-mean_y)**2
    res = s/(sqrt(s_x2*s_y2))
    # print(res, a, a2, b, b2)
    return res

In [18]:
x = []
y = []
for film in critics['Lisa Rose']:
    x.append(critics['Lisa Rose'][film])
    y.append(critics['Gene Seymour'][film])

# 11. TODO
print(pearson_corr(x, y)) # должно быть: 0.39605901719066977

0.39605901719066977


In [19]:
sim_pearson(critics, 'Lisa Rose', 'Gene Seymour')

np.float64(0.3960590171906697)

**Упражнение**  

Найдите в модуле scipy как вычисляется корреляция Пирсона и получите аналогичное значение (0.3960590171906697) как и с помощью вручную написанной функции.

In [30]:
from scipy.stats import pearsonr
pearsonr(x,y).statistic


np.float64(0.3960590171906697)

### Ранжирование критиков

После того как мы выбрали критерий для сравнения двух людей, можно начать искать для каждого человека подгруппу других людей с наиболее похожими вкусами.

Для этого в модуле **recommendations** реализована функция **topMatches()**. Функция возвращает список n наилучших соответствий для человека из словаря critics.

Соответственно, аргументы у функции:
* словарь критиков *critics\_dict*
* имя персоны *person*, для которой подбираем соответствия
* число *n* наилучших соответствий

Кроме этого, так как мы хотим использовать различные расстояния, то в качестве аргумента мы будем также передавать имя функции-расстояния (*similarity*).  

Пример вычисления топ-3 критиков похожих на Toby по корреляции Пирсона:

**Code 12:**

In [21]:
topMatches(critics, 'Toby', n=3, similarity=sim_pearson)

[(np.float64(0.9912407071619302), 'Lisa Rose'),
 (np.float64(0.924473451641905), 'Mick LaSalle'),
 (np.float64(0.8934051474415643), 'Claudia Puig')]

Как работает эта функция:
*  считаем схожесть интересующего нас критика с каждым другим критиком используя функцию **similarity**
*  сортируем результаты
*  возвращаем топ **n** результатов

**Code 13:**  
**Упражнение** Вычислите топ-3 критиков похожих на Toby по евклидову расстоянию (sim_distance):

In [22]:
# your code here

# 13. TODO

Выбор расстояния для подбора похожести зависит от конкретной задачи. Имеет смысл попробовать разные критерии и посмотреть, какой из них дает наилучший результат.

## Рекомендация фильмов (User-based подход)

Мы умеем находить людей с похожим мнением как у рассматриваемого человека. Но как получить конкретную рекомендацию с набором фильмов? Можно было бы просто посмотреть на то, какие фильмы понравились похожему человеку, и выбрать из них не просмотренные. Однако, такой способ не стабилен: можем, например, отобрать критика, которому по каким-то причинам понравился фильм, получивший негативные оценки от остальных.

Чтобы разрешить эту проблему, можно ранжировать сами фильмы, вычисляя взвешенную сумму оценок критиков. Мы будем брать оценку фильма у критика и умножать его на коэффициент похожести с тем, для которого мы строим рекомендацию. Затем полученный результат суммировать и нормировать.  

Идея такого подхода в том, что люди похожие на тебя вносят больший вклад в "рейтинг" фильма.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1x0QC1Blvr_ZugJ5BKPJdi2Bp30FvBxvL"
alt="IMAGE ALT TEXT HERE" width="500" border="0" /></a>

Этот подход реализован в модуле **recommendations** в функции **getRecommendations()**. Функция принимает на вход следующие аргументы:
* словарь критиков *critics\_dict*
* имя персоны *person*, для которой подбираем соответствия
* имя функции-критерия для сравнения

**Code 14:**

**Упражнение**

Получите рекомендации для ```Toby``` с помощью функции-критерия ```sim_pearson``` для сравнения:

In [23]:
# your code here

# 14. TODO

Как работает эта функция:
*  считаем схожесть пользователя с каждым другим пользователем
*  перебираем фильмы, которые смотрели другие пользователи, но не смотрел текущий пользователь
*  нормируем рейтинг фильмов на основе того насколько он понравился другим пользователям, и того насколько каждый из других пользователей похож на текущего пользователя
*  сортируем список

**Code 15:**

**Упражнение** Вычислите рекомендации для ```Toby``` с использованием ```sim_distance``` в качестве функции-критерия для сравнения:

In [24]:
# your code here

# 15. TODO

Мы получили ранжированный список фильмов, а также прогноз оценки, которую поставит Toby этим фильмам.

### Достоинства user-based подхода

* сравнительно простая реализация;
* не зависит от контекста;
* довольно точный, если достаточно данных;

### Недостатки user-based подхода

- фильмов много, каждый пользователь смотрит очень мало фильмов, поэтому матрица оценок будет разряженной (много пропущенных значений), это снижает точность предсказаний;

- точность рекомендаций зависит от количества ближайших соседей, которые мы используем для предсказания нашего пользователя, а значит время, которое мы тратим для предсказания, увеличивается одновременно с увеличением количества соседей (чем больше пользователей, тем медленнее в целом работает);

- для новых пользователей очень сложно делать предсказания, для новых фильмов также сложно помещать их в рекомендацию (термин, который характеризует проблему можно обозначить как [cold start recommender systems](https://en.wikipedia.org/wiki/Cold_start_(recommender_systems))).

## Коллаборативная фильтрация по сходству объектов (Item-based collaborative filtering)

Мы реализовали механизм выдачи рекомендаций таким образом, что для создания набора данных необходимы оценки, выставленные каждым
пользователем. Для нескольких тысяч людей это, возможно, и будет работать, но, когда речь идет о миллионах пользователей и товаров, сравнение каждого пользователя со всеми другими, а затем сравнение товаров, займет недопустимо много времени.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1Vd3ofIheUgmxDKLq72cVPTPyl5IyAxo0"
alt="IMAGE ALT TEXT HERE" width="500" border="0" /></a>


Техника, которую мы применяли до сих пор, называется **коллаборативной фильтрацией по схожести пользователей**. Альтернатива известна под названием **коллаборативная фильтрация по схожести образцов**.

Основная идея процедуры фильтрации по схожести образцов заключается в том, чтобы для каждого образца заранее вычислить большинство похожих на него. Тогда для выдачи рекомендаций алгоритму достаточно будет найти те образцы, которым пользователь выставил наивысшие оценки, а затем сформировать список образцов, максимально похожих на эти.

Отметим одно существенное отличие: хотя на первом шаге необходимо исследовать все данные, результаты сравнения образцов изменяются не так часто, как результаты сравнения пользователей.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1VAoSpaUcbabIFRWlXqrBFDs7LhIISkA5"
alt="IMAGE ALT TEXT HERE" width="700" border="0" /></a>

Чтобы сравнивать образцы, нужно первым делом построить полный набор данных о похожих образцах. В первую очередь, нам нужно переделать словарь предпочтений критиков таким образом, чтобы теперь был словарь фильмов и их оценок:
```python
film_dict = {
    'film_name_1':
    {'critic_1': 'score_1',
     'critic_2': 'score_2',
     ...
    },
    ...
}
```
Далее для каждого фильма посчитать наиболее похожие образцы и коэффициенты похожести для них. Для подсчета схожести будем использовать евклидово расстояние.

В модуле **recommendations** есть функция **calculateSimilarItems()**, которая делает сразу оба шага. На вход её нужно подать словарь с оценками критиков *critics* и число образцов (n), которые мы считаем наиболее похожими (остальные не выводим). Вызовите эту функцию с параметром ```n = 5```.

**Code 16:**

In [25]:
# 16. TODO
# itemsim =

In [26]:
pp.pprint(itemsim)

NameError: name 'itemsim' is not defined

Получили вложенный словарь с фильмами и их похожестью на другие фильмы.

Теперь мы готовы выдавать рекомендации, пользуясь словарем данных о схожести образцов. По аналогии с тем, что мы уже делали, необходимо получить список всех образцов, которым пользователь выставлял оценки, найти похожие и взвесить их с учетом коэффициентов подобия (умножить оценку на коэффициент подобия).

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1QIJpqC6zQkiQ-1-AzNSmvK4bc8fohsH1"
alt="IMAGE ALT TEXT HERE" width="500" border="0" /></a>

Этот механизм реализован в функции **getRecommendedItems()** из **recommendations**. На вход подается:
* словарь с оценками пользователя (в нашем случае это словарь критиков, *critics*)
* словарь с данными о схожести образцов, *itemsim*
* имя критика, для которого строим рекомендации

Получим рекомендации для пользователя Toby:


**Упражнение**

Получите список рекомендованных фильмов, используя данные о схожести ```itemsim``` и критика ```Toby```:

**Code 17:**

In [ ]:
## your code here

# 17. TODO


## Метрики качества для рекомендательных систем

Для оценки качества работы рекомендательных систем можно рассмотреть две специальные метрики **Hit Rate** (HR) и **Mean Reciprocal Rank** (MRR).

Определим как **попадание** (**hit**) - ситуацию, когда фильм был рекомендован пользователю в топ-N и пользователь посмотрел его.

**Hit Rate (HR)**

**Hit Rate (процент попаданий)** определяется как общее число попаданий, нормированное на количество пользователей.

$$
\text{HR} = \frac{1}{\text{# test users}} \sum_{\text{test users}}{\text{hit}}, \quad
$$

Чем больше процент попаданий, тем лучше будет наша система рекомендаций.

*Замечание*: в некотором смысле, Hit Rate представляет собой производное от классических ML метрик precision и recall.

**Mean Reciprocal Rank (MRR)**

Отличие метрики **Mean Reciprocal Rank** от предыдущей в том, что мы учитываем не общее количество попаданий, а обратное рангу (месту в списке рекомендаций) **первое попадание**. Например, если фильм, просмотренный пользователем (попадание), стоял на втором месте, обратное рангу будет равно $\frac{1}{2}$, а если бы стоял на третьем - $\frac{1}{3}$ и так далее.

$$
\text{MRR} = \frac{1}{\text{# test users}} \sum_{\text{test users}}{\frac{1}{\text{hit rank}}}
$$

Рассмотрим на примере. Toby получил следующий список рекомендаций:

**Упражнение**

Получите с помощью функции `getRecommendations` для критика Toby с помощью функции `sim_pearson` список рекомендаций:

**Code 18:**

In [ ]:
# Your code here
# 18. TODO

Допустим, после этого Toby посмотрел 5 фильмов и выставил им следующие оценки:


|Score|Film|
|----|--------------------|
|4.5 | The Departed |
|4   | The Night Listener |
|3.9 | Lady in the Water |
|3.7 | The Firm |
|3   | Just My Luck |

Как мы видим, только два фильма из списка рекомендаций попали в личный топ-3 Toby. Получается, что **попадание (hits)** у него 2.

**HR = $\dfrac{1}{1} * 2 = 2$**, так как выборка тестовых пользователей (test\_users) состоит из одного Toby.  

* **$\frac{1}{hit\_rank}$** для фильма The Night Listener - это $\dfrac{1}{2}$
* **$\frac{1}{hit\_rank}$** для фильма Lady in the Water - это $\dfrac{1}{3}$

Но фильм **The Night Listener** стоит выше в рейтинге Toby, чем **Lady in the Water**, поэтому:

**MRR = $\dfrac{1}{1}*\dfrac{1}{2} = \dfrac{1}{2}$**, так как, опять же, выборка тестовых пользователей (test\_users) состоит из одного Toby.  

## Бонус* : Рекомендация на данных MovieLens

В этом примере мы рассмотрим реальный набор данных с оценками фильмов, который называется MovieLens. Источник данных: http://grouplens.org/datasets/movielens/

Датасет представляет собой набор фильмов, которые 162541 пользователей сайта MovieLens оценивали по 5-бальной шкале. В наборе также есть дополнительная информация такая как описание фильмов тегами, но в нашем упражнении мы её использовать не будем для упрощения. Также, чтобы уменьшить время вычислений, будем использовать данный только 943 пользователей. Каждый пользователь в датасете оценил минимум 20 фильмов.

Загруженный датасет будет в следующем виде:

```python

preferences = {
    'user_id_1' : {
        'film_name_1' : rate,
        'film_name_2' : rate,
        ...
        'film_name_21' : rate,
    },
    ...
    'user_id_943' : {
        'film_name_1' : rate,
        'film_name_2' : rate,
        ...
        'film_name_30' : rate,
    }
}

```


Чтобы загрузить набор данных MovieLens, в модуле **recommendations** есть функция **loadMovieLens()**. Она возвращает словарь предпочтений фильмов для набора людей. Пользователи зашифрованы идентификатором. Для каждого пользователя имеется название
фильма, а также оценка, выставленная фильму данным пользователе.

**Code 19:**

In [ ]:
%%capture
!wget https://www.dropbox.com/sh/ob1qyfzqygkc6ph/AADg2G0By0EuRZCLR5gSAmP7a

In [ ]:
%%capture
!unzip AADg2G0By0EuRZCLR5gSAmP7a -d data

In [ ]:
preferences = loadMovieLens()

Выведем фильмы для пользователя с ID 87 и рейтинги этих фильмов

**Code 20:**

In [ ]:
pp.pprint(preferences['87'])

Посмотрим, какие рекомендации получим с помощью коллаборативной фильтрации по схожести пользователей. Напомним, что для этого реализована функция **getRecommendations()**, которая выдает рекомендации для данного пользователя по похожести на других и предположительную его оценку фильму.

Функция принимает на вход следующие аргументы:
* словарь пользователей, в этом случае это *preferences*
* имя персоны *person*, для которой подбираем соответствия
* имя функции-критерия для сравнения

Выведем первые топ-30 рекомендаций для пользователя с ID 87, для этого в квадратных скобочках укажем [0:30].

*Замечание*: может появиться предупреждение RuntimeWarning, это связано с тем, что теперь мы работаем с большой выборкой и при подсчете корреляции Пирсона может возникать деление на ноль. В этом случае мы присваиваем корреляции значение ноль.

**Code 21:**

In [ ]:
# Your code here

# 19-1. TODO

Интересно также сравнить полученный результат с рекомендациями по схожести образцов. Напомним, чтобы сравнить образцы, нужно в первую очередь перестроить наш набор данных из набора предпочтений пользователей в набор оценок фильмов. Для этого используем функцию **calculateSimilarItems()**. На вход её нужно подать словарь с оценками пользователей *preferences*. Кроме этого, мы передадим аргумент *n = 50*, так как хотим оставить только 50 наиболее похожих образцов в новом словаре.  

В этот раз мы работаем с большим набором данных и вычисления могут занять некоторое время, поэтому функция также будет выводить сообщение о том, сколько фильмов она уже просмотрела.

**Code 22:**

In [ ]:
# Your code here

# 19-2. TODO


# item_similarities =

Посмотрим на результат:

**Code 23:**

In [ ]:
item_similarities["What's Eating Gilbert Grape (1993)"]

Теперь получим топ-30 рекомендаций для пользователя с учетом похожести фильмов. Используем функцию **getRecommendedItems()**, которой на вход подается:
* словарь с оценками пользователя (*preferences*)
* словарь с данными о схожести фильмов, *item_similarities*
* ID пользователя, для которого строим рекомендации (для пользователя с id №87)

**Code 24:**

In [ ]:
# Your code here

# 20. TODO


Выведем список фильмов, которые пользователь с id `87` оценил максимальным баллом:



**Code 25:**

*Примечание*: `preferences['_some_user_id_']` - содержит значения рейтингов фильмов для пользователя '_some_user_id_'

In [ ]:
# Your code here
# 21. TODO

Сравним полученные результаты с теми, которые сформировала рекомендательная система.

Валидация для моделей рекомендательных систем устроена достаточно сложно.
Например, нам нужно забыть часть рейтингов фильмов для пользователя, а потом попытаться восстановить их с
помощью нашей модели.

В данном случае, мы можем оценить адекватность полученных рекомендаций, просмотрев списки фильмов. Однако, на текущем датасете, из-за размера выборки, такое сделать сложно.

Приведем примеры интерпретации:
* В случае использования расстояния, основанного на коррелции, модель рекомендует в первую очередь боевики
* В случае использование похожестей фильмов мы видим как общепризнанные шедевры, так и боевики
* В реальности пользователю нравятся фильмы, которые нравятся всем, приключенческие фильмы и боевики

С такой точки зрения оба списка рекомендаций кажутся адекватным, а то, что они не пересекаются связано с тем,
что есть много похожих друг на друга фильмов - и мы не можем рекомендовать сразу все такие фильмы пользователю.


Для рекомендательных систем есть специальные библиотеки, например, https://surprise.readthedocs.io/, однако их применение требует более глубокого знакомства с темой.

## Самостоятельная работа

TODO